In [3]:
#! python
#from pyFWI import *
from FWIFunctions import *
import sys, re, datetime

import pandas as pd
import numpy as np

### Test the old python code using the data provided. 

In [4]:
# Load the CSV file into a DataFrame
data = pd.read_csv('testBatch.csv') # data format: 1st line -- date format, latitude, ffmc, dmc, and dc values
                                    ## 2nd line --- temperature, relative humidity, wind speed, and rain
## initialize the relevant fire variables, i.e., ffmc, dmc, dc. 
## Calculation of ISI, BUI, FWI does not value of previous time step and therefore doesn't need to be initialized. 
N = data.shape[0]
ffmc = np.zeros(N+1)
dmc = np.zeros(N+1)
dc = np.zeros(N+1)
isi = np.zeros(N+1)
bui = np.zeros(N+1)
fwi = np.zeros(N+1)
    
ini_val = data.columns.tolist()
lat = float(ini_val[-4])
ffmc[0] = float(ini_val[-3]) # read in the initial value from the title of the data
dmc[0] = float(ini_val[-2])
dc[0] = float(ini_val[-1])

# Rename columns. Note that the initial value of ffmc, dmc, dc and fwi will now be replaced with 'ffmc','dmc','dc','fwi'
df = data.reset_index()
df.columns = ['Date', 'Temp', 'RH', 'Wind', 'Rain', 'Data','format','ffmc','dmc','dc','fwi']
df = df.drop(columns=['ffmc','dmc','dc','fwi'], axis=1)  ## to remove confusion, drop these columns. 


for ii in range(1,N+1):
    ffmc[ii] = FFMC(df['Temp'][ii-1],df['RH'][ii-1],df['Wind'][ii-1],df['Rain'][ii-1],ffmc[ii-1])#TEMP,RH,WIND,RAIN,FFMCPrev)
    dmc[ii] = DMC(df['Temp'][ii-1],df['RH'][ii-1],df['Rain'][ii-1],dmc[ii-1],lat,int(df.Date[ii-1][:2]))#TEMP,RH,RAIN,DMCPrev,LAT,MONTH)
    dc[ii] = DC(df['Temp'][ii-1],df['Rain'][ii-1],dc[ii-1],lat,int(df.Date[ii-1][:2]))#TEMP,RAIN,DCPrev,LAT,MONTH)
    isi[ii] = ISI(df['Wind'][ii-1],ffmc[ii])
    bui[ii] = BUI(dmc[ii],dc[ii])
    fwi[ii] = FWI(isi[ii],bui[ii])
    #print(ii,isi[ii],bui[ii],fwi[ii])

### Test the python code using the data provided by https://github.com/cffdrs/cffdrs_r/blob/main/tests/testthat/data/fwi_01.csv

In [10]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/xjliu/Downloads/fwi_01.csv') # data format: 1st line -- date format, latitude, ffmc, dmc, and dc values
                                    ## 2nd line --- temperature, relative humidity, wind speed, and rain
## initialize the relevant fire variables, i.e., ffmc, dmc, dc. 
## Calculation of ISI, BUI, FWI does not value of previous time step and therefore doesn't need to be initialized. 
N = 48
ffmc = np.zeros(N+1)
dmc = np.zeros(N+1)
dc = np.zeros(N+1)
isi = np.zeros(N+1)
bui = np.zeros(N+1)
fwi = np.zeros(N+1)
    
lat = 40
ffmc[0] = df['FFMC'][0]
dmc[0] = df['DMC'][0]
dc[0] = df['DC'][0]

# Rename columns. Note that the initial value of ffmc, dmc, dc and fwi will now be replaced with 'ffmc','dmc','dc','fwi'

for ii in range(1,N):
    ffmc[ii] = FFMC(df['TEMP'][ii],df['RH'][ii],df['WS'][ii],df['PREC'][ii],ffmc[ii-1])#TEMP,RH,WIND,RAIN,FFMCPrev)
    dmc[ii] = DMC(df['TEMP'][ii],df['RH'][ii],df['PREC'][ii],dmc[ii-1],lat,df['MON'][ii])#TEMP,RH,RAIN,DMCPrev,LAT,MONTH)
    dc[ii] = DC(df['TEMP'][ii],df['PREC'][ii],dc[ii-1],lat,df['MON'][ii])#TEMP,RAIN,DCPrev,LAT,MONTH)
    isi[ii] = ISI(df['WS'][ii],ffmc[ii])
    bui[ii] = BUI(dmc[ii],dc[ii])
    #bui[ii] = BUI(df['DMC'][ii],df['DC'][ii])
    fwi[ii] = FWI(isi[ii],bui[ii])

    #fwi[ii] = FWI(df['ISI'][ii],df['BUI'][ii])

    #print(ii,isi[ii],bui[ii],fwi[ii])

In [11]:
## compare the output of my code vs output of the R package.
(ffmc[1:48]-df['FFMC'][1:].values)/df['FFMC'][1:].values*100
(isi[1:48]-df['ISI'][1:].values)/df['ISI'][1:].values*100
(bui[1:48]-df['BUI'][1:].values)/df['BUI'][1:].values*100
(fwi[1:48]-df['FWI'][1:].values)/df['FWI'][1:].values*100
(dmc[1:48]-df['DMC'][1:].values)/dmc[1:48]*100

array([0.10607542, 0.06311165, 0.07061988, 0.0348743 , 0.04943997,
       0.03578398, 0.06992864, 0.02558942, 0.26751629, 0.20108399,
       0.23878341, 0.13809277, 0.14642446, 0.12396062, 0.11548526,
       0.07109918, 0.07787139, 0.0855051 , 0.06139586, 0.07066801,
       0.04841328, 0.0537754 , 0.04098148, 0.03374526, 0.04786482,
       0.14615517, 0.13325612, 0.12753532, 0.11029653, 0.10684039,
       0.09204118, 0.09932811, 0.22786322, 0.31670865, 0.29078571,
       0.38569722, 0.30243448, 0.2627172 , 0.34736879, 0.25775426,
       0.34853378, 0.46436482, 0.29323671, 0.45676629, 0.42692066,
       0.33242439, 0.30939861])

### Compare with the python code of FWI_xclim https://xclim.readthedocs.io/en/latest/_modules/xclim/indices/fire/_cffwis.html#cffwis_indices

In [13]:
import xclim
from xclim.testing import open_dataset
ds = open_dataset("/Users/xjliu/Downloads/daily_surface_cancities_1990-1993.nc")
ds = ds.assign(
     hurs=xclim.atmos.relative_humidity_from_dewpoint(ds=ds),
     tas=xclim.core.units.convert_units_to(ds.tas, "degC"),
     pr=xclim.core.units.convert_units_to(ds.pr, "mm/d"),
     sfcWind=xclim.atmos.wind_speed_from_vector(ds=ds)[0],
)
season_mask = xclim.indicators.atmos.fire_season(
        tas=ds.tas,
        method="WF93",
        freq="YS",
        # Parameters below are at their default values, but listed here for explicitness.
        temp_start_thresh="12 degC",
        temp_end_thresh="5 degC",
        temp_condition_days=3
    )
out_fwi = xclim.indicators.atmos.cffwis_indices(
        tas=ds.tas,
        pr=ds.pr,
        hurs=ds.hurs,
        sfcWind=ds.sfcWind,
        lat=ds.lat,
        season_mask=season_mask,
        overwintering=True,
        dry_start="CFS",
        prec_thresh="1.5 mm/d",
        dmc_dry_factor=1.2,
        # Parameters below are at their default values, but listed here for explicitness.
        carry_over_fraction=0.75,
        wetting_efficiency_fraction=0.75,
        dc_start=15,
        dmc_start=6,
        ffmc_start=85
    )

/Users/xjliu/miniconda3/lib/python3.9/site-packages/xclim/testing/utils.py:248: UserWarning: /Users/xjliu/Downloads/daily_surface_cancities_1990-1993.nc.md5 not accessible in remote repository. Unable to determine validity with upstream repo.
  warnings.warn(msg)
/Users/xjliu/miniconda3/lib/python3.9/site-packages/xclim/core/cfchecks.py:42: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
/Users/xjliu/miniconda3/lib/python3.9/site-packages/xclim/core/cfchecks.py:46: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
/Users/xjliu/miniconda3/lib/python3.9/site-packages/xclim/core/cfchecks.py:42: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
/Users/xjliu/miniconda3/lib/python3.9/site-packages/xclim/core/cfchecks.py:46: UserWarning: Variable has a non-conforming standard_name: Got `lwe_precipitation_rate`, expected `['precipitation_flux

In [14]:
## My code
N = 10
ffmc = np.zeros(N+1)
dmc = np.zeros(N+1)
dc = np.zeros(N+1)
isi = np.zeros(N+1)
bui = np.zeros(N+1)
fwi = np.zeros(N+1)
    
lat = 44.5
ffmc[0] = out_fwi[2][4,211]   # Use the output of xlim as the initial value (Aug 1) of FFMC, DMC, and DC for my code. 
dmc[0] = out_fwi[1][4,211]    # Pick Victoria and August 1. 
dc[0] = out_fwi[0][4,211]

## Pick Victoria, August 1 to Aug 6
ds2 = ds.sel(time=slice('1991-08-01','1991-08-06')) 
temp = ds2.tas[4,:]
prec = ds2.pr[4,:]
rh = ds2.hurs[4,:]
ws = ds2.sfcWind[4,:]

# Use temp, rh, wind, precip of current day, and FFMC, DMC, and DC of previous day. E.g. use temp etc. of Aug 2
# and FFMC,DMC,and DC of Aug 1. 

for ii in range(1,5):    
    ffmc[ii] = FFMC(temp[ii],rh[ii],ws[ii],prec[ii],ffmc[ii-1])#TEMP,RH,WIND,RAIN,FFMCPrev)
    dmc[ii] = DMC(temp[ii],rh[ii],prec[ii],dmc[ii-1],lat,8)#TEMP,RH,RAIN,DMCPrev,LAT,MONTH)
    dc[ii] = DC(temp[ii],prec[ii],dc[ii-1],lat,8)#TEMP,RAIN,DCPrev,LAT,MONTH)
    isi[ii] = ISI(ws[ii],ffmc[ii])
    bui[ii] = BUI(dmc[ii],dc[ii])
    fwi[ii] = FWI(isi[ii],bui[ii])

In [15]:
fwi

array([0.        , 3.34332293, 3.36009161, 3.68777158, 3.53304679,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [16]:
out_fwi[-1][4,211:216]

<xarray.DataArray 'fwi' (time: 5)> Size: 20B
array([7.6385612, 5.0258408, 5.888858 , 4.6721406, 4.012229 ],
      dtype=float32)
Coordinates:
    lon       float32 4B -123.2
    location  <U9 36B 'Victoria'
    lat       float32 4B 48.5
  * time      (time) datetime64[ns] 40B 1990-07-31 1990-08-01 ... 1990-08-04
Attributes:
    units:          1
    cell_methods:   tas: time: mean within days sfcWind: uas: time: mean with...
    history:        tas: \npr: \nsfcWind: uas: \nvas: \n[2024-11-07 11:30:43]...
    standard_name:  fire_weather_index
    long_name:      Fire weather index
    description:    Numeric rating of fire intensity.

In [96]:
out_fwi[2]

<xarray.DataArray 'ffmc' (location: 5, time: 1461)> Size: 29kB
array([[     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [83.07057,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan]],
      dtype=float32)
Coordinates:
    lon       (location) float32 20B -63.4 -73.4 -68.4 -106.7 -123.2
  * location  (location) <U9 180B 'Halifax' 'Montréal' ... 'Victoria'
    lat       (location) float32 20B 44.5 45.5 63.75 52.0 48.5
  * time      (time) datetime64[ns] 12kB 1990-01-01 1990-01-02 ... 1993-12-31
Attributes:
    units:          1
    cell_methods:   tas: time: mean within days sfcWind: uas: time: mean with...
    history:        tas: \npr: \nsfcWind: uas: \nvas: \n[2024-11-06 16:38:22]...
    standard_name:  fine_fuel_moisture_code
    long_name:      Fine fuel moisture code
    description:    Numeric rating of the average moisture content of litter ...